## Authentication DJANGO REST FRAMEWORK

Tipos:
 - Basic
 - Token
 - OAuth
 - JWT

### Setting the authentication scheme

## Basic Authentication

Only for testing not in production

### config/settings.py

In [ ]:
REST_FRAMEWORK = {
        'DEFAULT_AUTHENTICATION_CLASSES': [
         'rest_framework.authentication.BasicAuthentication',
         #'rest_framework.authentication.SessionAuthentication'
        ]
}

### products/views/py

In [ ]:
from rest_framework.permissions import IsAuthenticated

from rest_framework import generics
from .pagination import ProductLimitObjectPagination, ProductCursorPagination
from .serliaziers import ProductSerializer

class ProductListView(generics ListAPIView):
    queryset = Product.objects.all()
    serliazier_class = ProductSerializer
    pagination_class = ProductCursorPagination 
    permission_class = [
        IsAuthenticated
    ] #<----

### Base 64 Encode

https://www.base64encode.org/

### Postman

- Headers
- KEY : Authentication
- VALUE : Basic:carlos:clavesecreta
    - After Encode
    - Basic:YGZFEERTHS...===


## Token Authentication

### config/settings.py

In [ ]:
ISNSTALLED_APPS = {        
         'rest_framework.authtoken',

}

REST_FRAMEWORK = {
        'DEFAULT_AUTHENTICATION_CLASSES': [
         'rest_framework.authentication.TokenAuthentication',
        ]
}

python manage.py migrate

### Example use of token manually

- Postman
   - Headers:
   - KEY: Authorization:  Token:XFFHHTESSWEXS==

## Login Page

python mamange.py startapp user_app

### user_app/urls.py

In [ ]:
from django.urls import path
from rest_framework.authentication.views import obtain_auth_token

url_patterns = [
    path("login/", obtain_auth_token, namespace="login")
]

### config/urls.py

In [ ]:
url_patterns = [
    ...
    path("account", include("user_app.urls"))
]

### settings.py

In [ ]:
ISNSTALLED_APPS = [
    "user_app.apps.UserAppConfig",
    ...
]

### apps.py

In [ ]:
class UserAppConfig(AppConfig):
    default_autofield = "django.db.models.BigAutoField"
    name = "user_app"

### Postman

 - POST
 - BODY:
    - form-data:
        - KEY: 
            - username:
            - VALUE: carlos
        - KEY:
            - password:
            - VALUE: clavesecreta

- The return will be the token


## User Register

### user_app/serializers.py

In [ ]:
from django.contrib.auth.models import User
from rest_framework.serializers import ModelSerializer, ValidationError, CharField

class RegistrationSerializer(ModelSerializer):
    password2 = ChartField(style={
        "input_type":"password"}, 
        write_only=True
        )
    
    class Meta:
        model = User
        fields = ["username", "email", "password", "password2"]
        extra_kwargs = {
            "password" : {"write_only: True}
        }
    
    def save(self):
        password = self.validate_data("password")
        password2 = self.validate_data("password2")

        if password != password2:
            raise ValidationError({"error":"password y password2 deben ser iguales"})
        
        email = self.validate_data["email"]

        if User.objects.filter(email=email).exists():
            raise ValidationError({"errorr": "Ese correo ya esta registrado"})
        
        username = self.validate_data["username"]

        account = User(email=email, username=username)
        account.set_password(password)
        account.save()

        return account


### user_app/views.py

In [ ]:
from rest_framework.decorators import api_view
from rest_framework.response import Response
from user_app.serializers import RegistrationSerializer

@api_view(['POST', ]) ## Declara view basada en funciones
def registration_view(request):
    if request.method == "POST":
        serializer = RegistrationSerializer(data = request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        else:
            return Response(serializer.errors)

### user_app/urls.py

In [ ]:
url_patterns = [
    path("login", obtain_auth_token),
    path("register", registration_view)
]

## Creacion automatica del Token

### user_app/models.py

In [ ]:
from django.conf import settings
from django.db.models.signals import post_save
from django.dispatch import receiver
from rest_framework.authtoken.models import Token

@receiver(post_save, sender=settings.AUTH_USER_MODEL)
def create_auth_token(sender, instance=None, created=False, **kwargs):
    if created:
        Token.objects.create(user=instance)


### user_app/views.py

In [ ]:
from rest_framework.decorators import api_view
from rest_framework.response import Response
from user_app.serializers import RegistrationSerializer
from rest_framework.authtoken import Token

@api_view(['POST', ]) ## Declara view basada en funciones
def registration_view(request):
    if request.method == "POST":
        serializer = RegistrationSerializer(data = request.data)

        data = {}

        if serializer.is_valid():
            account = serializer.save()

            data["response"] = "Registro Exitoso"
            data["username"] = account.username
            data["email"] = account.email

            token = token.objects.get(user=account).key

            data["token"] = token

        else:
            data = serializer.errors

    return Response(data)

## Logout

### user_app/views.py

In [ ]:
@api_view(['POST', ])
def logout_view(request):
    if request.method == "POST":
        request.user.auth_token.delete()
        return Response(status=HTTP_200_OK)

### user_app/urls.py

In [ ]:
url_patterns = [
   ..
   path("logout/", logout_view)
]

## JWT

https://www.jwto.io

- Access Token: 5 minutes
- Refresh Token: 1 day
- Do not need the db to store it

Parts:
 - Header
 - Payload
 - Signature


Benefits:
- Do not require to access the db to get tokens

- Install 

In [ ]:
djangorestframework-simplejwt

### config/settings.py

In [ ]:
ISNSTALLED_APPS = {        
         'rest_framework.authtoken',

}

REST_FRAMEWORK = {
        'DEFAULT_AUTHENTICATION_CLASSES': [
         'rest_framework.simplejwt.authentication.JWTAuthentication',
        ]
}

SIMPLE_JWT = [
    "ROTATE_REFRESH_TOKENS": True
]

### user_app/urls.py 

In [ ]:
from rest_framework_simplejwt.views import TokenObtainPairView, TokenRefreshView

url_patterns = [
   ...
   path("login", obtain_auth_token),
   path("register", registration_view)
   path("api/token", TokenObtainPairView.as_view()),
   path("api/token/refresh", TokenRefreshView.as_view()),
]

### user_app/views.py

In [ ]:
from rest_framework_simpleJWT.tokens import RefreshToken
from rest_framework.decorators import api_view
from rest_framework.response import Response
from user_app.serializers import RegistrationSerializer
from rest_framework.authtoken import Token

@api_view(['POST', ]) ## Declara view basada en funciones
def registration_view(request):
    if request.method == "POST":
        serializer = RegistrationSerializer(data = request.data)

        data = {}

        if serializer.is_valid():
            account = serializer.save()

            data["response"] = "Registro Exitoso"
            data["username"] = account.username
            data["email"] = account.email

            jwt_tokens = RefreshToken.for_user(account)
            data["token"] = {
                "refresh": str(refresh_token),
                "access": str(refresh_token.access_token)
            }

        else:
            data = serializer.errors

    return Response(data)
    

class LoginView(APIView):
    def post(self, request):
        username = request.data.get('username')
        password = request.data.get('password')
        user = authenticate(username=username, password=password)
        if user:
            token, created = Token.objects.get_or_create(user=user)
            return Response({'token': token.key})
        return Response({'error': 'Credenciales inválidas'}, status=400)


### Probando JWT

In [ ]:
Postman

- account/api/token/

- POST
 - BODY:
    - form-data:
        - KEY: 
            - username:
            - VALUE: carlos
        - KEY:
            - password:
            - VALUE: clavesecreta

- The return will be the 2 Tokens 

    - Reresh Token
    - Access Token


- account/product/products/

- GET
 - HEADERS:
        - KEY: 
            - Authentication:
            - VALUE: Bearer "WESFVT.. Access Token

### Usando Refresh

http://localhost.../account/api/token/refresh/

In [ ]:
- POST
 - BODY:
    - x-www-form-urlencoded
        - KEY: refresh
        - VALUE: "SDFGX... Refresh Token


Returns a new access token, which shuold be used to get the products